In [ ]:
#assumptions that will break if we use MCTS for play traces in this code
#1. Turn wont be either 0 or 1
#2. Cant just copy the final two rows to extend round number to maxNoOfRounds

In [119]:
import pandas as pd
import pdb
pd.set_option('display.max_rows', 100)

In [59]:
data  = pd.read_csv("data/featureslogfile.txt", sep = '\t')

In [60]:
print(data.columns)

Index(['GameID', 'Player', 'Round', 'Turn', 'CurrentScore', 'SCORE',
       'SCORE_ADV', 'ORDINAL', 'OUR_TURN', 'HAS_WON', 'FINAL_ORD', 'ROUND',
       'CURSE', 'ESTATE', 'DUCHY', 'PROVINCE', 'COPPER', 'SILVER', 'GOLD',
       'CELLAR', 'CHAPEL', 'MOAT', 'HARBINGER', 'MERCHANT', 'VASSAL',
       'VILLAGE', 'WORKSHOP', 'BUREAUCRAT', 'GARDENS', 'MILITIA',
       'MONEYLENDER', 'POACHER', 'REMODEL', 'SMITHY', 'THRONE_ROOM', 'BANDIT',
       'COUNCIL_ROOM', 'FESTIVAL', 'LABORATORY', 'LIBRARY', 'MARKET', 'MINE',
       'SENTRY', 'WITCH', 'ARTISAN', 'PlayerCount', 'TotalRounds',
       'ActionScore', 'Win', 'Ordinal', 'FinalScore'],
      dtype='object')


In [61]:
#check number of games
print(data['GameID'].unique())

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [62]:
#note that Player field swaps round half way through the tournament when there are two players. Also the GameID starts
#at 2 for some reason
noGamesPerMatchUp = 25
def assignName(gameID, player, noGamesPerMatchUp):
    name = ""
    if gameID < (noGamesPerMatchUp + 2):
        if player == 0:
            name = 'BigMoneyWithGardens'
        else:
            name = 'DoubleWitch'
    else:
        if player == 1:
            name = 'BigMoneyWithGardens'
        else:
            name = 'DoubleWitch'
    return name

In [63]:
#add label for player types
data['PlayerName'] = data.apply(lambda df: assignName(df['GameID'], df['Player'], noGamesPerMatchUp), axis = 1)

In [66]:
print(data['GameID'].unique())

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [65]:
#we can only record events each turn so we need to use the final turn in each round for each player to determine their
#deck at the end of the round
print(data['Turn'].unique())
data = data[data['Turn'] == 1]
print(data['Turn'].unique())

[0. 1.]
[1.]


In [70]:
#extract columns of interest 
card_types_in_supply = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY',
                        'THRONE ROOM', 'WITCH', 'CURSE','WORKSHOP','PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types_BigMoneyWithGardens_vs_DoubleWitch = ['GARDENS', 'WITCH', 'CURSE', 'PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']

In [ ]:
#define columns of interest
cols = ['PlayerName', 'GameID', 'Round', 'CurrentScore']
cols = cols + card_types_BigMoneyWithGardens_vs_DoubleWitch

In [71]:
#extract all BigMoneyWithGardens play-traces
Traces_BigMoneyWithGardens = data[data['PlayerName'] == 'BigMoneyWithGardens'].loc[:, cols]
print(Traces_BigMoneyWithGardens)

               PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
2     BigMoneyWithGardens     2.0    0.0           3.0      0.0    0.0    0.0   
6     BigMoneyWithGardens     2.0    1.0           3.0      0.0    0.0    0.0   
10    BigMoneyWithGardens     2.0    2.0           3.0      0.0    0.0    0.0   
14    BigMoneyWithGardens     2.0    3.0           3.0      0.0    0.0    0.0   
18    BigMoneyWithGardens     2.0    4.0           3.0      0.0    0.0    0.0   
...                   ...     ...    ...           ...      ...    ...    ...   
4337  BigMoneyWithGardens    51.0   14.0          -1.0      0.0    0.0    4.0   
4341  BigMoneyWithGardens    51.0   15.0          -1.0      0.0    0.0    4.0   
4345  BigMoneyWithGardens    51.0   16.0           4.0      0.0    0.0    5.0   
4349  BigMoneyWithGardens    51.0   17.0          10.0      0.0    0.0    5.0   
4353  BigMoneyWithGardens    51.0   18.0          15.0      0.0    0.0    6.0   

      PROVINCE  DUCHY  ESTA

In [72]:
#some sense checks
print(Traces_BigMoneyWithGardens['WITCH'].unique())
print(Traces_BigMoneyWithGardens['GameID'].unique())

[0.]
[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51.]


In [73]:
#extract all DoubleWitch play-traces
Traces_DoubleWitch = data[data['PlayerName'] == 'DoubleWitch'].loc[:, cols]
print(Traces_DoubleWitch)

       PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
3     DoubleWitch     2.0    0.0           3.0      0.0    0.0    0.0   
7     DoubleWitch     2.0    1.0           3.0      0.0    0.0    0.0   
11    DoubleWitch     2.0    2.0           3.0      0.0    1.0    0.0   
15    DoubleWitch     2.0    3.0           3.0      0.0    2.0    0.0   
19    DoubleWitch     2.0    4.0           3.0      0.0    2.0    0.0   
...           ...     ...    ...           ...      ...    ...    ...   
4336  DoubleWitch    51.0   14.0          15.0      0.0    2.0    0.0   
4340  DoubleWitch    51.0   15.0          15.0      0.0    2.0    0.0   
4344  DoubleWitch    51.0   16.0          21.0      0.0    2.0    0.0   
4348  DoubleWitch    51.0   17.0          21.0      0.0    2.0    0.0   
4352  DoubleWitch    51.0   18.0          27.0      0.0    2.0    0.0   

      PROVINCE  DUCHY  ESTATE  GOLD  SILVER  COPPER  
3          0.0    0.0     3.0   0.0     1.0     7.0  
7          0.0 

In [75]:
#all play taces in a single data frame
traces = data.loc[:, cols]
print(traces)

               PlayerName  GameID  Round  CurrentScore  GARDENS  WITCH  CURSE  \
2     BigMoneyWithGardens     2.0    0.0           3.0      0.0    0.0    0.0   
3             DoubleWitch     2.0    0.0           3.0      0.0    0.0    0.0   
6     BigMoneyWithGardens     2.0    1.0           3.0      0.0    0.0    0.0   
7             DoubleWitch     2.0    1.0           3.0      0.0    0.0    0.0   
10    BigMoneyWithGardens     2.0    2.0           3.0      0.0    0.0    0.0   
...                   ...     ...    ...           ...      ...    ...    ...   
4345  BigMoneyWithGardens    51.0   16.0           4.0      0.0    0.0    5.0   
4348          DoubleWitch    51.0   17.0          21.0      0.0    2.0    0.0   
4349  BigMoneyWithGardens    51.0   17.0          10.0      0.0    0.0    5.0   
4352          DoubleWitch    51.0   18.0          27.0      0.0    2.0    0.0   
4353  BigMoneyWithGardens    51.0   18.0          15.0      0.0    0.0    6.0   

      PROVINCE  DUCHY  ESTA

In [108]:
#we need to make sure all games have the same number of rounds in them
gameLengths = traces.groupby(['GameID'])['Round'].max()
maxNoOfRounds = int(gameLengths.max())
print(maxNoOfRounds)

24


In [122]:
def copy_final_deck_at_game_end(group, roundMax, noPlayers):
    #This function repeatedly copies the final decks of two players at the game end, so that the game is extended to 
    #have roundMax rounds
    final_round = int(group['Round'].max())
    if roundMax == final_round:
        #in this case we dont need to extend the play trace
        return group
    else:
        final_row_copy = pd.concat([group.iloc[-noPlayers:]] * (roundMax - final_round), ignore_index=True)
        #we need to update the Round counter so that every other row it increments by one
        final_row_copy['Round'] = [final_round + 1 + i // 2 for i in range((roundMax - final_round)*2)]
        return pd.concat([group, final_row_copy], ignore_index=True)

In [126]:
extended_traces = traces.groupby('GameID').apply(copy_final_deck_at_game_end, maxNoOfRounds, noPlayers = 2).reset_index(drop = True)